Source: https://www.pinecone.io/learn/openai-gen-qa/

In [27]:
%pip install -qU openai pinecone datasets cohere tiktoken --upgrade

449.09s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip is available: 23.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Get the openai secret key
import getpass

OPENAI_API_KEY = getpass.getpass('Please enter your openai key: ')

In [4]:
from openai import OpenAI

# Get API key from top-right dropdown on OpenAI website
client = OpenAI(api_key=OPENAI_API_KEY)
MODEL = "gpt-4.1-mini"

In [ ]:
query = "who was the 12th person on the moon and when did they land?"

# Now query WITHOUT context
res = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ]
)

res.choices[0].message.content


'The 12th person to walk on the Moon was Eugene Cernan. He landed on the Moon during the Apollo 17 mission, which touched down on December 11, 1972.'

In [44]:
# First let's make it simpler to get answers
def complete(prompt):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "user", 
                "content": prompt
            }
        ]
    )
    return response.choices[0].message.content

query = (
    "Which training method should I use for sentence transformers when " +
    "I only have pairs of related sentences?"
)

complete(query)

'If you only have pairs of related sentences (i.e., positive pairs) and no explicit negative pairs or labels, a common and effective training method for sentence transformers is **training with a contrastive or triplet loss using semi-supervised or hard-negative mining strategies**, or leveraging **unsupervised/similarity-based losses**.\n\nHere are some common approaches you can take:\n\n### 1. **If you have only positive pairs:**\n\n- **Use a Contrastive Loss with Random Negatives:**  \n  You can treat each sentence pair as a positive pair and sample random sentences from your batch as negative pairs (in-batch negatives). This is a widely used approach in sentence embedding training.  \n  - Example: **InfoNCE / Multiple negatives ranking loss / NT-Xent loss**  \n  - Implementation: For each anchor-positive pair in the batch, all other sentences in the batch act as negatives. This encourages the model to pull related pairs close and push unrelated ones apart.\n\n- **Use Multiple Negat

In [ ]:
# Use OpenAI's text embedding model
embed_model = "text-embedding-3-small"

res = client.embeddings.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ],
    model=embed_model
)

# Vector embeddings for each document
res.data

[Embedding(embedding=[-0.0006560460315085948, 0.017824385315179825, 0.028480269014835358, -0.016606571152806282, -0.044727031141519547, -0.03373901546001434, 0.02431478723883629, -0.015485626645386219, 0.014766008593142033, -0.006019881926476955, 0.03415418043732643, -0.010289154015481472, 0.0047017354518175125, -0.006542297080159187, 0.0650423988699913, 0.07246000319719315, -0.004393821582198143, 0.013312933966517448, -0.026805773377418518, 0.03683890774846077, 0.004642920568585396, 0.04671981930732727, -0.021422475576400757, 0.03445863351225853, 0.012710945680737495, -0.02185148000717163, -0.04054770991206169, -0.02493753284215927, 0.05117591470479965, -0.06244070455431938, -0.018557842820882797, -0.041239649057388306, 0.019125234335660934, -0.03044538013637066, -0.040160223841667175, 0.05463561788201332, 0.06559595465660095, 0.003992496524006128, -0.049792032688856125, -0.04818673059344292, -0.00959029421210289, -0.01252412237226963, 0.054884716868400574, 0.009562617167830467, -0.00

In [19]:
# We have created two vectors (one for each sentence input)
len(res.data)

2

In [20]:
# We have created two 1536-dimensional vectors
len(res.data[0].embedding), len(res.data[1].embedding)

(1536, 1536)

In [21]:
# We can also get the vector for a single sentence
res.data[0].embedding

[-0.0006560460315085948,
 0.017824385315179825,
 0.028480269014835358,
 -0.016606571152806282,
 -0.044727031141519547,
 -0.03373901546001434,
 0.02431478723883629,
 -0.015485626645386219,
 0.014766008593142033,
 -0.006019881926476955,
 0.03415418043732643,
 -0.010289154015481472,
 0.0047017354518175125,
 -0.006542297080159187,
 0.0650423988699913,
 0.07246000319719315,
 -0.004393821582198143,
 0.013312933966517448,
 -0.026805773377418518,
 0.03683890774846077,
 0.004642920568585396,
 0.04671981930732727,
 -0.021422475576400757,
 0.03445863351225853,
 0.012710945680737495,
 -0.02185148000717163,
 -0.04054770991206169,
 -0.02493753284215927,
 0.05117591470479965,
 -0.06244070455431938,
 -0.018557842820882797,
 -0.041239649057388306,
 0.019125234335660934,
 -0.03044538013637066,
 -0.040160223841667175,
 0.05463561788201332,
 0.06559595465660095,
 0.003992496524006128,
 -0.049792032688856125,
 -0.04818673059344292,
 -0.00959029421210289,
 -0.01252412237226963,
 0.054884716868400574,
 0.009

In [22]:
from datasets import load_dataset

data = load_dataset('jamescalam/youtube-transcriptions', split='train')
data

README.md:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

0.01s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


youtube-transcriptions.jsonl:   0%|          | 0.00/79.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/208619 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'published', 'url', 'video_id', 'channel_id', 'id', 'text', 'start', 'end'],
    num_rows: 208619
})

In [23]:
data[0]

{'title': 'Training and Testing an Italian BERT - Transformers From Scratch #4',
 'published': '2021-07-06 13:00:03 UTC',
 'url': 'https://youtu.be/35Pdoyi6ZoQ',
 'video_id': '35Pdoyi6ZoQ',
 'channel_id': 'UCv83tO5cePwHMt1952IVVHw',
 'id': '35Pdoyi6ZoQ-t0.0',
 'text': 'Hi, welcome to the video.',
 'start': 0.0,
 'end': 9.36}

In [24]:
from tqdm.auto import tqdm

new_data = []

window = 20  # number of sentences to combine
stride = 4  # number of sentences to 'stride' over, used to create overlap

for i in tqdm(range(0, len(data), stride)):
    i_end = min(len(data)-1, i+window)
    if data[i]['title'] != data[i_end]['title']:
        # in this case we skip this entry as we have start/end of two videos
        continue
    text = ' '.join(data[i:i_end]['text'])
    # create the new merged dataset
    new_data.append({
        'start': data[i]['start'],
        'end': data[i_end]['end'],
        'title': data[i]['title'],
        'text': text,
        'id': data[i]['id'],
        'url': data[i]['url'],
        'published': data[i]['published'],
        'channel_id': data[i]['channel_id']
    })

  0%|          | 0/52155 [00:00<?, ?it/s]

In [25]:
new_data[0]

{'start': 0.0,
 'end': 74.12,
 'title': 'Training and Testing an Italian BERT - Transformers From Scratch #4',
 'text': "Hi, welcome to the video. So this is the fourth video in a Transformers from Scratch mini series. So if you haven't been following along, we've essentially covered what you can see on the screen. So we got some data. We built a tokenizer with it. And then we've set up our input pipeline ready to begin actually training our model, which is what we're going to cover in this video. So let's move over to the code. And we see here that we have essentially everything we've done so far. So we've built our input data, our input pipeline. And we're now at a point where we have a data loader, PyTorch data loader, ready. And we can begin training a model with it. So there are a few things to be aware of. So I mean, first, let's just have a quick look at the structure of our data.",
 'id': '35Pdoyi6ZoQ-t0.0',
 'url': 'https://youtu.be/35Pdoyi6ZoQ',
 'published': '2021-07-06 13:0

In [ ]:
from pinecone import Pinecone, ServerlessSpec

import os

PINECONE_API_KEY = getpass.getpass("Please enter your pinecone key: ")

# Initialize connection (get API key at app.pinecone.io):
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [32]:
index_name = "employee-handbook"
pc = Pinecone()  # This reads the PINECONE_API_KEY env var

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # Using the same vector dimensions as text-embedding-ada-002
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [33]:
# Connect to Index:
index = pc.Index(name=index_name)

In [34]:
# Describe the Index:
description = pc.describe_index(name=index_name)
print(description)

{'deletion_protection': 'disabled',
 'dimension': 1536,
 'host': 'employee-handbook-2321907.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'employee-handbook',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': False, 'state': 'Initializing'},
 'tags': None,
 'vector_type': 'dense'}


In [36]:
from tqdm.auto import tqdm
import datetime
import time
from time import sleep

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(new_data), batch_size)):
    # find end of batch
    i_end = min(len(new_data), i+batch_size)
    meta_batch = new_data[i:i_end]
    
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = client.embeddings.create(input=texts, model=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = client.embeddings.create(input=texts, model=embed_model)
                done = True
            except:
                pass
                
    # prepare vectors for upsert
    vectors = []
    for data, embedding in zip(meta_batch, res.data):
        vectors.append({
            "id": data['id'],
            "values": embedding.embedding,
            "metadata": {
                'start': data['start'],
                'end': data['end'], 
                'title': data['title'],
                'text': data['text'],
                'url': data['url'],
                'published': data['published'],
                'channel_id': data['channel_id']
            }
        })
    
    # Upsert to Pinecone
    index.upsert(vectors=vectors, namespace="ns1")

  0%|          | 0/487 [00:00<?, ?it/s]

In [59]:
res = client.embeddings.create(
    input=[query],
    model=embed_model
)

# retrieve from Pinecone
xq = res.data[0].embedding

# get relevant contexts (including the questions)
res = index.query(namespace="ns1", vector=xq, top_k=2, include_metadata=True)

# Print search results in a readable format
print("Search results:")
print("-" * 80)
for i, match in enumerate(res['matches'], 1):
    print(f"\nMatch {i} (Score: {match['score']:.3f})")
    print(f"ID: {match['id']}")
    print("\nMetadata:")
    print(f"  Title: {match['metadata']['title']}")
    print(f"  Time: {match['metadata']['start']:.1f}s - {match['metadata']['end']:.1f}s") 
    print(f"  URL: {match['metadata']['url']}")
    print(f"  Published: {match['metadata']['published']}")
    print("\nText:")
    print("  " + match['metadata']['text'])

Search results:
--------------------------------------------------------------------------------

Match 1 (Score: 0.669)
ID: pNvujJ1XyeQ-t418.88

Metadata:
  Title: Today Unsupervised Sentence Transformers, Tomorrow Skynet (how TSDAE works)
  Time: 418.9s - 568.4s
  URL: https://youtu.be/pNvujJ1XyeQ
  Published: 2021-11-24 16:24:24 UTC

Text:
  pairs of related sentences you can go ahead and actually try training or fine-tuning using NLI with multiple negative ranking loss. If you don't have that fine. Another option is that you have a semantic textual similarity data set or STS and what this is is you have so you have sentence A here, sentence B here and then you have a score from from 0 to 1 that tells you the similarity between those two scores and you would train this using something like cosine similarity loss. Now if that's not an option and your focus or use case is on building a sentence transformer for another language where there is no current sentence transformer you can use

In [61]:
limit = 3750

def retrieve(query):
    res = client.embeddings.create(
        input=[query], 
        model=embed_model
    )

    # retrieve from Pinecone
    xq = res.data[0].embedding

    # get relevant contexts
    res = index.query(namespace="ns1", vector=xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    
    # Initialize prompt with all contexts
    prompt = (
        prompt_start +
        "\n\n---\n\n".join(contexts) + 
        prompt_end
    )
    
    # If total length exceeds limit, reduce contexts one by one
    for i in range(len(contexts)-1, 0, -1):
        if len("\n\n---\n\n".join(contexts[:i])) < limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i]) +
                prompt_end
            )
            break
            
    return prompt

# First we retrieve relevant items from Pinecone
query_with_contexts = retrieve(query)
print(query_with_contexts)

Answer the question based on the context below.

Context:
pairs of related sentences you can go ahead and actually try training or fine-tuning using NLI with multiple negative ranking loss. If you don't have that fine. Another option is that you have a semantic textual similarity data set or STS and what this is is you have so you have sentence A here, sentence B here and then you have a score from from 0 to 1 that tells you the similarity between those two scores and you would train this using something like cosine similarity loss. Now if that's not an option and your focus or use case is on building a sentence transformer for another language where there is no current sentence transformer you can use multilingual parallel data. So what I mean by that is so parallel data just means translation pairs so if you have for example a English sentence and then you have another language here so it can it can be anything I'm just going to put XX and that XX is your target language you can fine

In [62]:
# Then we complete the context-infused query
print(complete(query_with_contexts))

If you only have pairs of related sentences, you should go ahead and try training or fine-tuning your sentence transformer using Natural Language Inference (NLI) data with multiple negative ranking loss. This supervised method leverages pairs of related sentences effectively and is one of the common approaches to fine-tuning sentence transformers when labeled data in that format is available.
